In [1]:
# The Datasets are not classied, they are separeted in two dirrent files. 
# we will merge them and make a labeled data directory

from __future__ import print_function
import pandas as pd
import shutil
import os
import sys

labels = pd.read_csv(r'D:\Datasets\Food dataset\train.csv')

train_dir =r'D:\Datasets\Food dataset\train_images\train_images'
_ = r"D:\Datasets\Food dataset\food with label\_"
if not os.path.exists(_):
    os.mkdir(_)

for filename, class_name in labels.values:
    
    if not os.path.exists(_ + str(class_name)):
        os.mkdir(_ + str(class_name))
    src_path = train_dir + '/'+ filename 
    dst_path = _ + str(class_name) + '/' + filename 
    try:
        shutil.copy(src_path, dst_path)
        
    except IOError as e:
        print('Unable to copy file {} to {}'.format(src_path, dst_path))
    except:
        print('When try copy file {} to {}, unexpected error: {}'
              .format(src_path, dst_path, sys.exc_info()))
print("sucessful")

In [2]:
# importing essential files
import tensorflow as tf
from tensorflow import keras
from keras .layers import Dense, Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir = (r'D:\Datasets\Food dataset\food with label')
train_image_generator = ImageDataGenerator(
    height_shift_range = 0.3,
    rescale = 1./255,
    rotation_range = 32,
    horizontal_flip = True, 
    fill_mode = 'nearest'
)
x_train = train_image_generator.flow_from_directory(
        train_dir,
        target_size=(128,128),
        class_mode='categorical',
        shuffle=True)


Found 9323 images belonging to 61 classes.


In [76]:
# creating the list of class names. 
import pandas as pd
classes = []
labels = pd.read_csv(r'D:\Datasets\Food dataset\train.csv')
for filename, class_name in labels.values:
    if class_name not in classes:
        classes.append(class_name)

len(classes)

61

In [4]:
# manually making a Sequential model using convolutiona d MaxPooling layers
model = keras.Sequential([
    keras.layers.Conv2D(32,(3,3), padding ='same', input_shape=(128,128,3),
                        activation=('relu')),
    keras.layers.Conv2D(32,(3,3), activation ='relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64,(3,3), activation =('relu')),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64,(3,3), activation =('relu')),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(61, activation='softmax'),    
    ])
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

              

In [5]:
# LEts Enable GPU computation 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("using GPU")
    except RuntimeError as e:
        print(e)

In [6]:
# First Model Not so much effecctive :(
with tf.device('/GPU:0'):
      history = model.fit( x_train, epochs =3, )

Epoch 1/3
292/292 [==============================] - 202s 690ms/step - loss: 3.8597 - accuracy: 0.0872
Epoch 2/3
292/292 [==============================] - 227s 779ms/step - loss: 3.8103 - accuracy: 0.0904
Epoch 3/3
292/292 [==============================] - 225s 769ms/step - loss: 3.6828 - accuracy: 0.1159


In [7]:
#   Lets Go for Transfer Learnig. We will use the Inception Model and Imagenet weights !


inception_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top= False, weights='imagenet',input_shape=(128,128,3))

for layer in inception_model.layers:
      layer.trainable = False

model2 = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(61, activation='softmax')
])    

inception_model = keras.Sequential([inception_model, model2])
inception_model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

inception_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2, 2, 2048)        21802784  
_________________________________________________________________
sequential_1 (Sequential)    (None, 61)                499773    
Total params: 22,302,557
Trainable params: 499,773
Non-trainable params: 21,802,784
_________________________________________________________________


In [8]:
inception_model.fit(x_train, epochs = 8 )

Epoch 1/8
292/292 [==============================] - 134s 452ms/step - loss: 7.4417 - accuracy: 0.2674
Epoch 2/8
292/292 [==============================] - 130s 445ms/step - loss: 6.8534 - accuracy: 0.3274
Epoch 3/8
292/292 [==============================] - 130s 445ms/step - loss: 7.2109 - accuracy: 0.3505
Epoch 4/8
292/292 [==============================] - 129s 440ms/step - loss: 7.0110 - accuracy: 0.3603
Epoch 5/8
292/292 [==============================] - 129s 440ms/step - loss: 6.8162 - accuracy: 0.3797
Epoch 6/8
292/292 [==============================] - 129s 443ms/step - loss: 6.9227 - accuracy: 0.3897
Epoch 7/8
292/292 [==============================] - 130s 447ms/step - loss: 6.9105 - accuracy: 0.3934
Epoch 8/8
292/292 [==============================] - 132s 453ms/step - loss: 6.8074 - accuracy: 0.4049


In [9]:
# prepairing the test data
test_img_path = r'D:\Datasets\Food dataset\test_images'
test_image_generator = ImageDataGenerator(
    rescale = 1./255
)
x_test = test_image_generator.flow_from_directory(
        test_img_path,
        target_size=(128,128),)
        # class_mode='categorical',)


Found 484 images belonging to 1 classes.


In [16]:
# Save the model for futr
inception_model.save(r"C:\Users\kiit\Desktop")

INFO:tensorflow:Assets written to: C:\Users\kiit\Desktop\assets


In [17]:
model.save(r"C:\Users\kiit\Desktop")

INFO:tensorflow:Assets written to: C:\Users\kiit\Desktop\assets


In [46]:
# lets make Prediction 
prediction = inception_model.predict(x_test)

In [81]:
import numpy as np
class_type = [np.argmax(element) for element in prediction]


In [110]:
x = []
for i in range(0,484):
      y = classes[class_type[i]]
      x.append(y)

x = pd.DataFrame(x)
x

,0
0,bread-whole-wheat
1,sauce-savoury
2,mixed-salad-chopped-without-sauce
3,mixed-nuts
4,salad-leaf-salad-green
...,...
479,cucumber
480,mixed-nuts
481,avocado
482,mayonnaise


In [114]:
# lets merge the wo file and save the result to a csv file 

test_csv = pd.read_csv(r"D:\Datasets\Food dataset\test.csv")
result = pd.concat([test_csv, x], axis=1)

In [126]:
result

,ImageId,0
0,90e63a2fde.jpg,bread-whole-wheat
1,a554d1ca8d.jpg,sauce-savoury
2,48317e8ee8.jpg,mixed-salad-chopped-without-sauce
3,79528df667.jpg,mixed-nuts
4,6d2f2f63f5.jpg,salad-leaf-salad-green
...,...,...
479,843b8695df.jpg,cucumber
480,584490f483.jpg,mixed-nuts
481,d5df1ab5ec.jpg,avocado
482,79e3e01b88.jpg,mayonnaise
